In [1]:
! pip3 install U pygame_widgets --user

In [2]:
import pygame


class Card(pygame.sprite.Sprite):
    
    
    def __init__(self, rank, suit, x=0, y=0):
        pygame.sprite.Sprite.__init__(self)
        self.rank = rank
        self.rect = pygame.Rect(x,y, 75, 140)
        self.suit = suit
        rank_to_value = dict(zip(['two','three','four','five','six','seven','eight', 'nine', 'ten', 'jack', 'queen', 'king', 'ace']
,[2,3,4,5,6,7,8,9,10,10,10,10,11]))
        self.value = rank_to_value[self.rank]
        self.front = False
        self.x = x
        self.y = y
        self.clicked = False
        self.image = pygame.Surface([75, 140]) 
        pygame.image.load('back.svg')
        pygame.draw.rect(self.image,'red',pygame.Rect(100, 100, 100, 200)) 
        self.image = pygame.transform.scale(pygame.image.load('back.svg').convert(), (75, 140)), (self.x, self.y)


    def img(self):
        if self.front == False: 
            self.image = 'back.svg'
        if self.front == True:
            if self.value<10 or self.rank=='ten': 
                self.image =  f'{self.value}_of_{self.suit}.png'
            else: 
                self.image = f'{self.rank.lower()}_of_{self.suit}.png'

    
    def update(self, x, y):
        self.rect = pygame.Rect(x,y,75,140)
        self.x = x
        self.y = y

    
    def __str__(self):
        return f'{self.rank} of {self.suit}'

    
    def __repr__(self):
        return f'{self.rank, self.suit, self.value}'

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.8)


In [3]:
from random import shuffle 


class Deck:
    
    def __init__(self):
        ranks = ['two','three','four','five','six','seven','eight','nine','ten','jack','queen','king','ace']
        suits = ['clubs', 'diamonds', 'hearts', 'spades']
        values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10 , 11]
        cards = []
        for suit in suits:
            for rank in ranks:
                cards.append(Card(rank,suit))
        self.cards = cards

    
    def __str__(self): 
        return f'{self.cards}, {len(self.cards)}]'
    
    
    def shuffle(self):
        '''Shuffles cards in deck '''
        shuffle(self.cards)
    
    
    def get_top_card(self):
        '''Returns the top card in the deck'''
        return self.cards.pop(0)


In [11]:
import pygame
import sys

pygame.init()

screen = pygame.display.set_mode((1400, 700))
screen.fill("dark green")
running = True
pygame.display.flip()
pygame.display.set_caption("Solitaire")

deck = Deck()
deck.shuffle()

text = pygame.font.SysFont('Times New Roman',40).render('START', True, 'black')
buttonx = 1400/2-70
buttony = 700/2+40
pygame.draw.rect(screen,'red',[buttonx,buttony,140,40]) 
screen.blit(text ,(buttonx,buttony)) 
pygame.display.flip()

stacks = []
clicked = False

while running:

    for event in pygame.event.get():
        
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            x,y = pygame.mouse.get_pos()
            if pygame.Rect(buttonx, buttony, 140, 40).collidepoint(x,y):
                
                screen.fill("dark green")
                
                for i in range(4):
                    screen.blit(pygame.transform.scale(pygame.image.load('place.png').convert(), (75, 140)), (50,150*i+50))
                screen.blit(pygame.transform.scale(pygame.image.load('back.svg').convert(), (75, 140)), (1250, 50))
                deck_pos = pygame.Rect(1250, 50, 75, 140)
                
                stacks = []

                for i in range(1,8): #rows
                    stack = []
                    
                    for t in range(0,i): #columns
                        x = 1400/9*i
                        y = (t)*75
                        card = deck.get_top_card()
                        card = Card(card.rank, card.suit, x ,y)
                        stack.append(card)
                        
                        if i-1 == t: #last card in stack
                            card.front = True
                        card.img()
                        screen.blit(pygame.transform.scale(pygame.image.load(card.image).convert(), (75, 140)), (x,y))
                        pygame.display.flip()
                    stacks.append(stack)    
                stacks.append([])
                cards =dict.fromkeys(range(1,8))
                for i in range(1,9):
                    cards[i]=(stacks[i-1])
                
            if deck_pos.collidepoint(x,y):
                card = deck.get_top_card()
                card = Card(card.rank, card.suit, 1250 ,300)
                card.img()
                screen.blit(pygame.transform.scale(pygame.image.load(card.image).convert(), (75, 140)), (1250, 300))
                cards[8].append(card)
            for stack in cards.values(): 
                for card in stack: 
                    if card.front == True and card.rect.collidepoint(x,y):
                        if card.clicked == False: 
                            pygame.draw.rect(screen, 'yellow', [card.x-2.5, card.y-2.5, 80,145])
                            screen.blit(pygame.transform.scale(pygame.image.load(card.image).convert(), (75, 140)), (card.x, card.y))
                            pygame.display.flip() 
                            clicked = card
                            pygame.time.wait(10)
                            card.clicked = True
                        else:
                            #card.rect.move_to(card.x, card.y+75))
                            card.update(clicked.x,clicked.y+75)
                            screen.blit(pygame.transform.scale(pygame.image.load(card.image).convert(), (75, 140)), (card.x, card.y))
                            #check for stacks and move to different stack maybe make a function
                            pygame.display.flip()
                            card.clicked = False
                            count = 0
        if event.type == pygame.QUIT:
            running = False  

pygame.quit()